<a href="https://colab.research.google.com/github/Titoffifee/CP_Dagestan/blob/main/transport.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.metrics import make_scorer, f1_score, recall_score

df = pd.read_csv('drive/MyDrive/transport/train.csv')

In [20]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35756 entries, 0 to 35770
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Место     35756 non-null  object
 1   Улица     35756 non-null  object
 2   Дорога    35756 non-null  object
 3   Вид ДТП   35756 non-null  object
 4   Час пик   35756 non-null  int64 
 5   Ночь      35756 non-null  int64 
 6   Зима      35756 non-null  int64 
 7   Пятница   35756 non-null  int64 
 8   Выходные  35756 non-null  int64 
dtypes: int64(5), object(4)
memory usage: 2.7+ MB


In [5]:
y_labels = ['Погибло детей', 'Ранено детей', 'Погибло', 'Ранено']
delete = ['id', 'Километр', 'Метр', 'Дом']

In [15]:
df = df[df['Ранено'] <= 5]
train, y0 = df.drop(delete + y_labels, axis=1), df[y_labels]

In [17]:
train['Вид ДТП'] = train['Вид ДТП'].fillna('Неизвестно')
train['Улица'] = train['Улица'].fillna('На шоссе')
train['Дорога'] = train['Дорога'].fillna('В городе')

In [18]:
from datetime import time, date

# 7:40-9:30 и 18:00-20:20 - час пик
# 23:00 - 6:00 - ночь

ph0, ph1, ph2, ph3 = time(7, 40, 0), time(9, 30, 0), time(18, 0, 0), time(20, 20, 0)
night0, night1 = time(23, 0, 0), time(6, 0, 0)

def GetGoodTime(tm):
  ans = pd.Series([0] * 2, index=['Час пик', 'Ночь'])
  tm = [int(i) for i in tm.split(':')]
  t = time(hour=tm[0], minute=tm[1], second=tm[2])
  if ph0 <= t <= ph1 or ph2 <= t <= ph3:
    ans['Час пик'] = 1
  if t >= night0 or t <= night1:
    ans['Ночь'] = 1
  return ans

def GetGoodDate(dt):
  ans = pd.Series([0] * 3, index=['Зима', 'Пятница', 'Выходные'])
  dt = [int(i) for i in dt.split('.')]
  dt = date(dt[2], dt[1], dt[0])
  if dt.month == 12 or dt.month <= 2:
    ans['Зима'] = 1
  if dt.weekday() == 4:
    ans['Пятница'] = 1
  elif dt.weekday() > 4:
    ans['Выходные'] = 1
  return ans

train_time = train['Время'].apply(GetGoodTime)
train_date = train['Дата'].apply(GetGoodDate)
train = train.drop(['Время', 'Дата'], axis=1)

train = pd.concat([train, train_time, train_date], axis=1)

In [19]:
test = pd.read_csv('drive/MyDrive/transport/test.csv')
test = test.drop(delete, axis=1)
test['Вид ДТП'] = test['Вид ДТП'].fillna('Неизвестно')
test['Улица'] = test['Улица'].fillna('На шоссе')
test['Дорога'] = test['Дорога'].fillna('В городе')
test_time = test['Время'].apply(GetGoodTime)
test_date = test['Дата'].apply(GetGoodDate)
test = test.drop(['Время', 'Дата'], axis=1)
test = pd.concat([test, test_time, test_date], axis=1)

In [10]:
obj = ['Место', 'Вид ДТП', 'Улица', 'Дорога']
max_value = dict()
model = CatBoostClassifier(silent=True, cat_features=obj, random_seed=0, 
                           loss_function='Logloss')
for name in y_labels:
  max_value[name] = y0[name].max()

index_name = dict()
index_name_list = []
for name in y_labels:
  index_name[name] = [name + str(i) for i in range(1, max_value[name]+1)]
  index_name_list += index_name[name]

In [ ]:
def CreateY(ser):
  ans = pd.Series(dtype=int)
  for name in y_labels:
    r = pd.Series([0] * max_value[name], index=index_name[name], dtype=int)
    for i in range(1, ser[name] + 1):
      r[name + str(i)] = 1
    ans = pd.concat([ans, r])
  return ans

def CreateRes(ser):
  ans = pd.Series([0] * 4, index=y_labels)
  for name in y_labels:
    r = 0
    for i in range(1, max_value[name] + 1):
      if ser[name + str(i)] == 1:
        r = i
    ans[name] = r
  return ans

y = y0.apply(CreateY, axis=1)
y

Тестирование:

In [ ]:
seeds = [0]
result = []
for seed in seeds:
  print(seed, 'START')
  X_train, X_test, Y_train, Y_test = train_test_split(train, y, test_size=0.35, random_state=seed)
  Y_pred = Y_test.copy()
  print('pred work end')
  for name in y_labels:
    print(name)
    for i in range(1, max_value[name] + 1):
      ind, last_ind = name + str(i), name + str(i-1)
      if i == 1:
        data_train = X_train
        data_test = X_test
      else:
        data_train = pd.concat([X_train, Y_train[last_ind]], axis=1)
        data_test = pd.concat([X_test, Y_pred[last_ind]], axis=1)
      model.fit(data_train, Y_train[ind])
      Y_pred[ind] = model.predict(data_test)
      print(ind + ' predict end')
  print('Start creating result')
  y_true_corr = Y_test.apply(CreateRes, axis=1)
  y_pred_corr = Y_pred.apply(CreateRes, axis=1)
  score = 0
  for name in y_labels:
    score += recall_score(y_true_corr[name], y_pred_corr[name], average='macro')
  print(seed, 'OK')
  result.append(score / 4)

In [ ]:
result

[0.2843858823650961]

Получение готового ответа:

In [12]:
p = pd.read_csv('drive/MyDrive/transport/solve.csv')
p = p[y_labels].apply(CreateY, axis=1)
result = pd.read_csv('drive/MyDrive/transport/solve.csv')

def CreateResOneObject(ser):
  ans = -1
  for i in range(len(ser)):
    if ser.iloc[i] == 1:
      ans = i
  return ans + 1

In [13]:
for name in y_labels:
  for i in range(1, max_value[name] + 1):
    ind, last_ind = name + str(i), name + str(i-1)
    # Добавим связь с предыдущем слоем
    if i == 1:
      data_train = train
      data_test = test
    else:
      data_train = pd.concat([train, y[last_ind]], axis=1)
      data_test = pd.concat([test, p[last_ind]], axis=1)
    print(ind, 'WORK')
    model.fit(data_train, y[ind])
    p[ind] = model.predict(data_test)
  print(name, 'predict OK')
  result[name] = p[index_name[name]].apply(CreateResOneObject, axis=1)
  print(name, 'transform OK')
result

Погибло детей1 WORK
Погибло детей predict OK
Погибло детей transform OK
Ранено детей1 WORK
Ранено детей2 WORK
Ранено детей3 WORK
Ранено детей4 WORK
Ранено детей predict OK
Ранено детей transform OK
Погибло1 WORK
Погибло2 WORK
Погибло3 WORK
Погибло predict OK
Погибло transform OK
Ранено1 WORK
Ранено2 WORK
Ранено3 WORK
Ранено4 WORK
Ранено5 WORK
Ранено predict OK
Ранено transform OK


,id,Погибло,Погибло детей,Ранено,Ранено детей
0,490078911,0,0,0,0
1,490055448,0,0,0,0
2,490054440,0,0,0,0
3,490037847,0,0,0,0
4,490037049,0,0,0,0
...,...,...,...,...,...
6783,490014641,0,0,0,0
6784,490033115,0,0,0,0
6785,490008221,0,0,0,0
6786,490119585,0,0,0,0


In [14]:
result.to_csv('drive/MyDrive/transport/ans.csv', index=False)